In [ ]:
import ipywidgets.widgets as widgets
from time import sleep
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.camera import get_images_and_cameras
from ballsbot.servos import get_controls
from ballsbot.utils import keep_rps
from ballsbot.lidar import Lidar
from ballsbot.session import start_rospy_session
from ballsbot.utils import run_as_thread
from ballsbot.imu import IMU_Threaded
from ballsbot.tracking import Tracker
from ballsbot.odometry import Odometry
from ballsbot.controller import link_controller

In [ ]:
start_rospy_session('a_car')

images = []
for image, camera in get_images_and_cameras():
    images.append(image)
hbox = widgets.HBox(images)
display(hbox)

# lidar_image = widgets.Image(format='jpeg', width=600, height=600)
# display(lidar_image)

track_image = widgets.Image(format='jpeg', width=800, height=800)
display(track_image)

controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

lidar = Lidar()

In [ ]:
def lidar_run():
    lidar.auto_update_lidar_cloud(fps=4)

run_as_thread(lidar_run)

In [ ]:
imu = IMU_Threaded()

In [ ]:
link_controller(controller)

In [ ]:
odometry = Odometry(imu, get_controls()['throttle'])
tracker = Tracker(imu, lidar, odometry, max_distance=5., fps=4, fix_pose_with_lidar=False) # FIXME

def tracker_run():
    tracker.start()
run_as_thread(tracker_run)

In [ ]:
def update_track():
    sleep(2)
    ts = None
    while True:
        ts = keep_rps(ts, fps=1)
        tracker.update_picture(track_image, only_nearby_meters=6.)

run_as_thread(update_track)

In [ ]:
# from ballsbot.utils import join_all_theads
# join_all_theads()

In [ ]:
# import json
# with open('/home/jumper/readings.json', 'w') as the_file:
#     the_file.write(json.dumps(tracker.readings, indent=4))

In [ ]:
# for it in odometry.readings:
#     print('teta {:+0.04f}, dteta {:+0.04f}, dteta_test {:+0.04f}, w_z {:+0.04f}'.format(it['teta'], it.get('dteta', 0.), it.get('dteta_test', 0.), it.get('w_z', 0.)))

In [ ]:
# import json
# with open('/home/jumper/o-readings.json', 'w') as the_file:
#     the_file.write(json.dumps(odometry.readings, indent=4))

In [ ]:
# tracker.poses_to_a_file('/home/jumper/poses.json')